In [1]:
%reset -f 
from __future__ import absolute_import, print_function
import time
import numpy as np
import pyopencl as cl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import signal
from matplotlib import rcParams
from scipy import io
import os
from scipy import signal

import locale
os.environ["PYTHONIOENCODING"] = "utf-8"


In [2]:
# Widening the screen
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

rcParams['figure.dpi'] = 600
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'
rcParams["mathtext.fontset"] = 'stix'
rcParams['animation.embed_limit'] = 2**128
plt.rc('font', size=16)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=16)   # fontsize of the figure title

In [3]:
kd=0.36         # Dissociation rate constant
k3=6.7          # intensity conversion factor
k1=1.35         #recruitment rate coefficient
tildeb=60       #self-triggering rate
b=tildeb/k3
rho=15          # stability: default value=5.0 set as estimated half-saturation concentration of DNA
k2=2.0          # half-saturation constant : threshold concentration
beta=17.8*1.1   # maximal reduction of intensity per hour (estimated)
Du=0.17*0.05    # diffusion for DNA-bound protein
Dv=0.17         # diffusion for free protein
cv=0.016*6.7    #constant speed up calculation

In [4]:
foldname="rho" + str(rho) + "_v25"
if not os.path.exists(foldname):
    os.makedirs(foldname)

In [5]:
length    = 100      #50#512.0# 50.0      # 2.5 Length of the physical landscape   
n         = 512      # Size of the 2D grid
endtime   = 10000 
dT        = 0.01    #0.01       # 0.02     #calculate time step
nplot     = 200      #200 #100
#nsteps    = np.ceil(endtime/nplot/dT).astype(int)    # number of time steps
LogTimeImages=np.logspace(np.log10(1),np.log10(endtime),nplot+1) #log not interger, okay?
nstepsARRAY=np.diff(LogTimeImages)/dT
nstepsARRAY=nstepsARRAY.astype(int)

### Precalculations
dX = 0.2 #length/n      # Spatial step size
dY = 0.2 #length/n      # Spatial step size
Grid_Width  = n
Grid_Height = n
# %% Reporting in the simulation on the console
print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

 Current grid dimensions: 512 x 512 cells



In [6]:
# Setting up the OpenCL context
#DeviceNr = 2   # 0 = GTX 960M
DeviceNr = 0    # 0= Quadro K5200
platform = cl.get_platforms()
Devices  = platform[0].get_devices()  # 0 = GPU
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)
mf       = cl.mem_flags # Memory flags are set
print(" Compute Device: %s\n" % Devices[DeviceNr].name)

 Compute Device: Apple M2 Ultra



In [7]:
for kk in range(1,10):
    u0= 0.5 # initial DNA-bound protein concentration
    u= u0+  (np.random.rand(n*n)-0.5)*0.02   # u ~(0,0.5)
    v0= 25.0 # initial free-protein concentration
    v=v0 + (np.random.rand(n*n)-0.5)*0.5 # v~(27.5,28.5)
    u_host   = u.astype(np.float32)
    u_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=u_host)
    v_host   = v.astype(np.float32)
    v_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=v_host)

    with open('SpatialFunctions_iPy.cl', 'r',encoding='utf-8') as myfile:
     SpatialFunctions = myfile.read()
    
    # List of parameters for the OpenCL kernel. Seperate with comma without spaces
    # Setting up the parameters for the Kernel
    PassVars="kd,k3,k1,b,k2,rho,cv,beta,Du,Dv,dX,dY,dT,Grid_Width,Grid_Height"
    PassVals=eval(PassVars)
    PassVars=PassVars.split(',')
    Params=""
    
    for ii in range(len(PassVals)):
        Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"
    
    ComputeCode = """
    ///////////////////////////////////////////////////////////////////////////////
    // Simulation kernel
    ///////////////////////////////////////////////////////////////////////////////   
    
    __kernel void SimulationKernel (__global float* u, __global float* v)
    {
        size_t current  = get_global_id(0);
        size_t row      = floor((float)current/(float)Grid_Width);
        size_t column   = current%Grid_Width;
    
            if (row > 0 && row < Grid_Width-1 && column > 0 && column < Grid_Height-1)
                {
                  
                    float dudt =  cv*(-kd*u[current] + (k1*u[current]+b)*v[current] - rho*k2*beta*u[current]/(k2+u[current]+v[current])) + Du*d2_dxy2(u); 
                    float dvdt =  -cv*(-kd*u[current] + (k1*u[current]+b)*v[current] - rho*k2*beta*u[current]/(k2+u[current]+v[current])) + Dv*d2_dxy2(v); 
                  
                 u[current] = u[current]+dudt*dT;
                 v[current] = v[current] +dvdt*dT; 
                } 
            // HANDLE Boundaries
            else 
                {
                  PeriodicBoundaries(u);
                  PeriodicBoundaries(v);
                 //NeumannBoundaries(u);
                 //NeumannBoundaries(v);
                 
                }
    } // End SimulationKernel
    """
    
    program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

    from ipywidgets import FloatProgress
    from IPython.display import display
    # Setting up a progress bar for the simulation
    print("Progress :");
    PB = FloatProgress(min=0, max=nplot); display(PB) 
    Us=np.zeros((Grid_Width, Grid_Height, nplot))
    Vs=np.zeros((Grid_Width, Grid_Height, nplot))

    # Set up simulation parameters
    global_size = u_host.shape

    # Start the timer:
    start_time = time.time()
    
    # Starting the loop
    for ii in range(1,nplot):
        # The simulation  
        for jj in range(1,nstepsARRAY[ii]):  
            program.SimulationKernel(queue, global_size, None, u_g, v_g)

        # Get the data from the GPU
        cl.enqueue_copy(queue, u_host, u_g)
        cl.enqueue_copy(queue, v_host, v_g)

        # We store the state of the system for <NumPlot> different times.
        Us[:,:,ii] = u_host.reshape(Grid_Width, Grid_Height)
        Vs[:,:,ii] = v_host.reshape(Grid_Width, Grid_Height)

        PB.value += 1 # signal to increment the progress bar

    # Determining the time that we used for the simulation
    elapsed_time = time.time() - start_time    
    print(" Simulation took      : %1.1f (s)" % (elapsed_time))
    io.savemat( foldname +'/sklaw_data'+str(kk)+'.mat', {'B': Us, 'Time': LogTimeImages})
    print("completed!")
    

Progress :


FloatProgress(value=0.0, max=200.0)

AGX: exceeded compiled variants footprint limit


 Simulation took      : 137.1 (s)
completed!
Progress :


FloatProgress(value=0.0, max=200.0)

...

 Simulation took      : 136.7 (s)
completed!
Progress :


FloatProgress(value=0.0, max=200.0)

 Simulation took      : 139.0 (s)
completed!
Progress :


FloatProgress(value=0.0, max=200.0)

 Simulation took      : 138.9 (s)
completed!
